In [22]:
import cv2
import numpy as np
import pyvisgraph as vis
from geopandas import GeoSeries
from shapely.geometry import Polygon, Point, LineString

In [23]:
img = cv2.imread('pictures/shapes3.jpg')
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

ret,thresh = cv2.threshold(gray,80,255,0)
contours,hierarchy = cv2.findContours(thresh, 1, 2)
coin = []
for cnt in contours:
    x1,y1 = cnt[0][0]
    approx = cv2.approxPolyDP(cnt, 0.01*cv2.arcLength(cnt, True), True)
    coin.append(approx)
    if len(approx) == 4:
        x, y, w, h = cv2.boundingRect(cnt)
        ratio = float(w)/h
        img = cv2.drawContours(img, [cnt], -1, (0,255,255), 3)
    else:
        img = cv2.drawContours(img, [cnt], -1, (0,255,255), 3)
        
cv2.imshow("Shapes", img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [24]:
start = vis.Point(str(10), str(10))
target = vis.Point(str(350), str(350))

listPoints = []
for i in range(len(coin)):
    obstacle = []
    for j in range(len(coin[i])):
        obstacle.append(vis.Point(coin[i][j][0][0], coin[i][j][0][1]))
    obstacle.append(vis.Point(coin[i][0][0][0], coin[i][0][0][1]))
    listPoints.append(obstacle)
    
graph = vis.VisGraph()
graph.build(listPoints)
shortest_path = graph.shortest_path(start, target)
    
print(shortest_path)

100%|████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 87.55it/s]

[Point(10.00, 10.00), Point(112.00, 148.00), Point(229.00, 271.00), Point(299.00, 399.00), Point(350.00, 350.00)]


In [9]:
for i in range(len(shortest_path ) -1):
    cv2.line(img, (int(shortest_path[i].x), int(shortest_path[i].y)), (int(shortest_path[ i +1].x), int(shortest_path[i+1].y)),
             (255, 0, 0), thickness=5)

NameError: name 'shortest_path' is not defined

In [14]:
listCenters = []
for i in coin:
    M = cv2.moments(i)
    if M['m00'] != 0:
        cx = int(M['m10']/M['m00'])
        cy = int(M['m01']/M['m00'])
        cv2.circle(img, (cx, cy), 7, (0, 0, 255), -1)
        listCenters.append([cx,cy])

In [15]:
margin = 30
marged_coin = []
for i in range(len(coin)):
    center = listCenters[i]
    new_coin = []
    for j in range(len(coin[i])):
        vect = (coin[i][j][0] - center)/np.linalg.norm(coin[i][j][0] - center)
        new_coin.append(coin[i][j][0]+ margin*vect)
    marged_coin.append(new_coin)


In [16]:
start = vis.Point(str(10), str(10))
target = vis.Point(str(350), str(350))

listPoints = []
for i in range(len(marged_coin)):
    obstacle = []
    for j in range(len(marged_coin[i])):
        obstacle.append(vis.Point(marged_coin[i][j][0], marged_coin[i][j][1]))
    obstacle.append(vis.Point(marged_coin[i][0][0], marged_coin[i][0][1]))
    listPoints.append(obstacle)
    
graph = vis.VisGraph()
graph.build(listPoints)
shortest_path = graph.shortest_path(start, target)
    
print(shortest_path)

100%|███████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 112.92it/s]

[Point(10.00, 10.00), Point(96.01, 162.83), Point(140.97, 227.61), Point(350.00, 350.00)]


In [17]:
for i in range(len(shortest_path ) -1):
    cv2.line(img, (int(shortest_path[i].x), int(shortest_path[i].y)), (int(shortest_path[ i +1].x), int(shortest_path[i+1].y)),
             (0, 255, 0), thickness=5)

cv2.imshow("Shapes", img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [38]:
def detect_obstacle(img, margin, show):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    ret,thresh = cv2.threshold(gray,80,255,0)
    contours,hierarchy = cv2.findContours(thresh, 1, 2)
    coin = []
    for cnt in contours:
        x1,y1 = cnt[0][0]
        approx = cv2.approxPolyDP(cnt, 0.01*cv2.arcLength(cnt, True), True)
        coin.append(approx)
        if show:
            img = cv2.drawContours(img, [cnt], -1, (0,255,255), 3)
            
    listCenters = []
    for i in coin:
        M = cv2.moments(i)
        if M['m00'] != 0:
            cx = int(M['m10']/M['m00'])
            cy = int(M['m01']/M['m00'])
            cv2.circle(img, (cx, cy), 7, (0, 0, 255), -1)
            listCenters.append([cx,cy])

    marged_coin = []
    for i in range(len(coin)):
        center = listCenters[i]
        new_coin = []
        for j in range(len(coin[i])):
            vect = (coin[i][j][0] - center)/np.linalg.norm(coin[i][j][0] - center)
            new_coin.append(coin[i][j][0]+ margin*vect)
        marged_coin.append(new_coin)
    
    for i in range(len(marged_coin)):
        for j in range(len(marged_coin[i])):
            img = cv2.circle(img, (int(marged_coin[i][j][0]), int(marged_coin[i][j][0])), 7, (0, 0, 255), -1)

    return marged_coin

In [39]:
def make_path(img, start, goal, obstacle, show):
    listPoints = []
    for i in range(len(obstacle)):
        obst = []
        for j in range(len(obstacle[i])):
            obstacle.append(vis.Point(obstacle[i][j][0], obstacle[i][j][1]))
        obst.append(vis.Point(obstacle[i][0][0], obstacle[i][0][1]))
        listPoints.append(obst)

    graph = vis.VisGraph()
    graph.build(listPoints)
    shortest_path = graph.shortest_path(start, target)

    if show:
        for i in range(len(shortest_path ) -1):
            cv2.line(img, (int(shortest_path[i].x), int(shortest_path[i].y)), (int(shortest_path[ i +1].x), int(shortest_path[i+1].y)),
                     (0, 255, 0), thickness=5)

    return img, shortest_path

In [40]:
start = vis.Point(str(10), str(10))
target = vis.Point(str(350), str(350))

img = cv2.imread('shapes3.jpg')
coin = detect_obstacle(img, 30, True)
img, shortest = make_path(img, start, target, coin, True)

cv2.imshow("Shapes", img)
cv2.waitKey(0)
cv2.destroyAllWindows()

100%|████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<?, ?it/s]
